# [Hands-On] Fine-tuning LLaMA for Question Answering: A Practical Guide

- Author: Sangkeun Jung (hugmanskj@gmail.com)

> Educational Purpose

**Copyright**: All rights reserved

## Overview

This notebook demonstrates how to fine-tune LLaMA 2 for Question Answering tasks using the SQuAD dataset. We'll cover the complete process from data preparation to model evaluation, using efficient training techniques like LoRA (Low-Rank Adaptation) and 4-bit quantization.

Key learning objectives:

* Understanding how to prepare QA data for instruction fine-tuning
* Implementing efficient fine-tuning using LoRA
* Working with quantized models for memory efficiency
* Evaluating fine-tuned models on QA tasks


## Environment Setup

First, we need to install the required packages. Each package serves a specific purpose in our fine-tuning pipeline:

- `accelerate`: Enables distributed training and optimization
- `peft`: Provides parameter-efficient fine-tuning methods
- `bitsandbytes`: Implements 4-bit quantization
- `transformers`: Core library for working with transformer models
- `trl`: Supports training language models with reinforcement learning
- `datasets`: Helps manage and process datasets

Let's install these dependencies:

In [3]:
!pip install bitsandbytes==0.46.1
!pip install trl==0.12.1


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
  Attempting uninstall: trl
    Found existing installation: trl 0.4.7
    Uninstalling trl-0.4.7:
      Successfully uninstalled trl-0.4.7

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


## Import Required Libraries

## Required Libraries

We'll import various libraries that are essential for our fine-tuning process:

- `torch`: The core deep learning framework
- `datasets`: For handling the SQuAD dataset
- `transformers`: Provides the model and tokenizer
- `peft`: For implementing LoRA
- `trl`: Contains the SFTTrainer for supervised fine-tuning
- Other utility libraries for data manipulation

These imports set up the foundation for our fine-tuning pipeline.

In [4]:
import os
import torch
from datasets import Dataset, load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import pandas as pd

## Data Preparation with SQuAD

### Understanding SQuAD Dataset
The Stanford Question Answering Dataset (SQuAD) is a reading comprehension dataset that contains:
- Questions created by crowdworkers
- Corresponding passages from Wikipedia articles
- Answers extracted from these passages

### Data Format
We'll convert our QA pairs into an instruction format suitable for LLaMA 2. Each example will contain:
1. Context: The relevant passage
2. Question: The query to be answered
3. Answer: The ground truth from the dataset

### Instruction Format Example:
```text
[INST]
### Context
{passage text}
### Question
{question text}
[/INST]
{answer text}
```

In [5]:
from datasets import load_dataset

dataset = load_dataset("squad")
train_df = dataset['train'].to_pandas()
validation_df = dataset['validation'].to_pandas()

In [6]:
train_df.iloc[:10]

id                     title  \
0  5733be284776f41900661182  University_of_Notre_Dame   
1  5733be284776f4190066117f  University_of_Notre_Dame   
2  5733be284776f41900661180  University_of_Notre_Dame   
3  5733be284776f41900661181  University_of_Notre_Dame   
4  5733be284776f4190066117e  University_of_Notre_Dame   
5  5733bf84d058e614000b61be  University_of_Notre_Dame   
6  5733bf84d058e614000b61bf  University_of_Notre_Dame   
7  5733bf84d058e614000b61c0  University_of_Notre_Dame   
8  5733bf84d058e614000b61bd  University_of_Notre_Dame   
9  5733bf84d058e614000b61c1  University_of_Notre_Dame   

                                             context  \
0  Architecturally, the school has a Catholic cha...   
1  Architecturally, the school has a Catholic cha...   
2  Architecturally, the school has a Catholic cha...   
3  Architecturally, the school has a Catholic cha...   
4  Architecturally, the school has a Catholic cha...   
5  As at most other universities, Notre Dame's st...   
6  As at most other universities, Notre Dame's st...   
7  As at most other universities, Notre Dame's st...   
8  As at most other universities, Notre Dame's st...   
9  As at most other universities, Notre Dame's st...   

                                            question  \
0  To whom did the Virgin Mary allegedly appear i...   
1  What is in front of the Notre Dame Main Building?   
2  The Basilica of the Sacred heart at Notre Dame...   
3                  What is the Grotto at Notre Dame?   
4  What sits on top of the Main Building at Notre...   
5  When did the Scholastic Magazine of Notre dame...   
6   How often is Notre Dame's the Juggler published?   
7  What is the daily student paper at Notre Dame ...   
8  How many student news papers are found at Notr...   
9  In what year did the student paper Common Sens...   

                                             answers  
0  {'text': ['Saint Bernadette Soubirous'], 'answ...  
1  {'text': ['a copper statue of Christ'], 'answe...  
2  {'text': ['the Main Building'], 'answer_start'...  
3  {'text': ['a Marian place of prayer and reflec...  
4  {'text': ['a golden statue of the Virgin Mary'...  
5  {'text': ['September 1876'], 'answer_start': [...  
6         {'text': ['twice'], 'answer_start': [441]}  
7  {'text': ['The Observer'], 'answer_start': [598]}  
8         {'text': ['three'], 'answer_start': [126]}  
9          {'text': ['1987'], 'answer_start': [908]}

### Data Formatting for Instruction Fine-tuning
We need to convert our QA pairs into an instruction format that LLaMA 2 can understand. The format includes:

- Context: The passage containing the answer
- Question: The query we want the model to answer
- Answer: The ground truth answer from the dataset

In [7]:
# Define the conversion function
def convert_to_instruction_format(row):
    return f"""
    <s>
    [INST]
    ### Context
    {row['context']}
    ### Question
    {row['question']}
    [/INST]
    {row['answers']['text'][0]}
    </s>"""

# Apply the function to the DataFrame
train_df['text'] = train_df.apply(convert_to_instruction_format, axis=1)
validation_df['text'] = validation_df.apply(convert_to_instruction_format, axis=1)

In [8]:
print( train_df.iloc[0].text )


    <s>
    [INST]
    ### Context
    Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.
    ### Question
    To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
    [/INST]
    Saint Bernadette Soubirous
    </s>


In [9]:
train_df = train_df.iloc[:1000]

In [10]:
dataset = Dataset.from_pandas(train_df[['text']])
dataset

Dataset({
    features: ['text'],
    num_rows: 1000
})

## Model Configuration and Setup

### Base Model Selection
We're using the LLaMA 2 7B Chat model (`NousResearch/Llama-2-7b-chat-hf`) as our foundation. This model is:
- Optimized for dialogue and instruction following
- Pre-trained on a large corpus of text
- Suitable for fine-tuning on specific tasks

### Quantization Setup
To make training feasible on consumer GPUs, we implement 4-bit quantization:

```python
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=False,
)
```

### Quantization Configuration

In [11]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, TaskType

model_id = "NousResearch/Llama-2-7b-hf"
tokenizer_id = model_id

# 8bit quantization 설정
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.float16,
    bnb_8bit_use_double_quant=True,
)

# 모델 로딩
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map='auto',
    trust_remote_code=True,
    torch_dtype=torch.float16,
)

# 토크나이저 설정
tokenizer = AutoTokenizer.from_pretrained(tokenizer_id)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

### LoRA Configuration Section

LoRA is a parameter-efficient fine-tuning technique that significantly reduces memory requirements while maintaining performance.



In [12]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    # Llama-2 전용 target modules
    target_modules=[
        "q_proj", "v_proj", "k_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    bias="none",

    use_rslora=False,  # RSLoRA 사용 안함
    use_dora=False,    # DoRA 사용 안함
    init_lora_weights=True,  # LoRA 가중치 초기화
)
model = get_peft_model(model, lora_config)



### Model and Tokenizer Initialization

Load and configure the model and tokenizer:



### Training Configuration

Set up training parameters:



In [13]:
training_arguments = TrainingArguments(
    output_dir="./results",

    ## epoch & gradient accumulation
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,

    ## optimzer
    weight_decay=0.001,
    max_grad_norm=0.3,

    ## Learning weight Schedule
    learning_rate=2e-4,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",

    ## Loging
    save_steps=0,
    logging_steps=25,

    ## Datatype
    fp16=False,
    bf16=False,

    ## Data Handling
    group_by_length=True,  # Grouping by length for efficient batching
)

### SFT training settings




In [15]:
max_seq_length = None  # Max sequence length
packing = False  # Pack short sequences together

#Setting up the fine-tuning trainer with the specified model, dataset, tokenizer, and training arguments
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,

    ## data handling
    train_dataset=dataset,
    dataset_text_field="text",  # Specifying which dataset field to use for text
    max_seq_length=max_seq_length,  # Setting the maximum sequence length
    packing=packing,  # Enabling packing for efficiency

    ## peft
    peft_config=lora_config,

    ## training args
    args=training_arguments,
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:403: UserWarning: You passed a processing_class with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `processing_class.padding_side = 'right'` to your code.
  warnings.warn(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


## Training
Execute the training:

In [16]:
trainer.train()

TrainOutput(global_step=63, training_loss=1.0875753372434587, metrics={'train_runtime': 135.1244, 'train_samples_per_second': 7.401, 'train_steps_per_second': 0.466, 'total_flos': 1.1411747714924544e+16, 'train_loss': 1.0875753372434587, 'epoch': 1.0})

## Model Evaluation

For testing, we need to format our prompts without the answers:

In [17]:
# Define the conversion function
def convert_to_prompt_format(row):
    return f"""
    [INST]
    ### Context
    {row['context']}
    ### Question
    {row['question']}
    [/INST]
    """
validation_df['prompt'] = validation_df.apply(convert_to_prompt_format, axis=1)

In [18]:
print( validation_df.iloc[0].prompt )


    [INST]
    ### Context
    Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.
    ### Question
    Which NFL team represented the AFC at Super Bowl 50?
    [/INST]
    


In [19]:
logging.set_verbosity(logging.CRITICAL)

prompt = validation_df.iloc[0].prompt

pipe = pipeline(task="text-generation",
                model=model,
                tokenizer=tokenizer, max_length=300)

result = pipe(prompt)

print("-"*50)
print(prompt)
print("-"*50)
print("[PREDICTED]")
print(result[0]['generated_text'].split("[/INST]")[1])
print("="*50)
print("[ACTUAL]")
print(validation_df.iloc[0].answers['text'][0])

--------------------------------------------------

    [INST]
    ### Context
    Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.
    ### Question
    Which NFL team represented the AFC at Super Bowl 50?
    [/INST]
    
--------------------------------------------------

## Conclusion

In this tutorial, we've covered:

1. How to prepare QA data for fine-tuning LLaMA 2
2. Implementation of efficient training techniques (LoRA and 4-bit quantization)
3. The complete fine-tuning pipeline for question-answering tasks
4. Model evaluation and inference

The fine-tuned model can now better understand and answer questions based on given contexts, making it suitable for various QA applications.

Key takeaways:

- LoRA enables efficient fine-tuning by updating only a small number of parameters
- 4-bit quantization makes it possible to fine-tune large models on consumer hardware
- Proper prompt formatting is crucial for instruction fine-tuning
- The resulting model can be used in a pipeline for easy inference